In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_ase_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,Webinar on Careers in Chemical Engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0,0,NaN,0.3818,Positive,4
1,1,Webinar on Careers in Chemical Engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0,0,NaN,0.3818,Positive,4
2,2,"Amrita School of Engineering, Bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0,0,1.0,-0.4278,Negative,2
3,3,"Amrita School of Engineering, Bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0,0,1.0,-0.4278,Negative,2
4,4,"Amrita School of Engineering, #Bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0,0,1.0,0.9080,Positive,5
5,5,"Amrita School of Engineering, #Bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0,0,1.0,0.9080,Positive,5
6,6,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0,3,NaN,0.0000,Neutral,3
7,7,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0,3,NaN,0.0000,Neutral,3
8,8,Webinar on Importance of Mathematics for Engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0,0,NaN,0.3612,Positive,4
9,9,Webinar on Importance of Mathematics for Engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0,0,NaN,0.3612,Positive,4


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [6]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,Webinar on Careers in Chemical Engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000
1,Webinar on Careers in Chemical Engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000
2,"Amrita School of Engineering, Bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000
3,"Amrita School of Engineering, Bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000
4,"Amrita School of Engineering, #Bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000
5,"Amrita School of Engineering, #Bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000
6,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358
7,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358
8,Webinar on Importance of Mathematics for Engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000
9,Webinar on Importance of Mathematics for Engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000


In [7]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [8]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [9]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [10]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [11]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig..."
1,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig..."
2,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu..."
3,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu..."
4,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer..."
5,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer..."
6,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team..."
7,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team..."
8,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000,0.000000,Webinar on Importance of Mathematics for Engin...,"[webinar, importance, mathematics, engineering..."
9,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000,0.000000,Webinar on Importance of Mathematics for Engin...,"[webinar, importance, mathematics, engineering..."


In [12]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig...","[(webinar, NN), (careers, NNS), (chemical, VBP..."
1,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig...","[(webinar, NN), (careers, NNS), (chemical, VBP..."
2,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu...","[(amrita, JJ), (school, NN), (engineering, NN)..."
3,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu...","[(amrita, JJ), (school, NN), (engineering, NN)..."
4,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer...","[(amrita, JJ), (school, NN), (engineering, NN)..."
5,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer...","[(amrita, JJ), (school, NN), (engineering, NN)..."
6,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team...","[(dance, NN), (hidden, JJ), (language, NN), (s..."
7,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team...","[(dance, NN), (hidden, JJ), (language, NN), (s..."
8,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000,0.000000,Webinar on Importance of Mathematics for Engin...,"[webinar, importance, mathematics, engineering...","[(webinar, NN), (importance, NN), (mathematics..."
9,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000,0.000000,Webinar on Importance of Mathematics for Engin...,"[webinar, importance, mathematics, engineering...","[(webinar, NN), (importance, NN), (mathematics..."


In [13]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig...","[(webinar, NN), (careers, NNS), (chemical, VBP...","[webinar, careers, engineering, brighter, futu..."
1,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,Positive,4,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig...","[(webinar, NN), (careers, NNS), (chemical, VBP...","[webinar, careers, engineering, brighter, futu..."
2,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, principals, p..."
3,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,Negative,2,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, principals, p..."
4,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, offers, engin..."
5,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,Positive,5,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, offers, engin..."
6,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team...","[(dance, NN), (hidden, JJ), (language, NN), (s...","[dance, language, soul, body, pc, teammedia, a..."
7,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,Neutral,3,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team...","[(dance, NN), (hidden, JJ), (language, NN), (s...","[dance, language, soul, body, pc, teammedia, a..."
8,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.000000,0.000000,0.000000,Webinar on Importance of Mathematics for Engin...,"[webinar, importance, mathematics, engineering...","[(webinar, NN), (importance, NN), (mathematics...","[webinar, importance, mathematics, engineering..."
9,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,Positive,4,0.0

In [14]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,webinar
1,careers
2,engineering
3,brighter
4,future
5,school
6,coimbatore
7,pm
8,registration
9,bengaluru


In [15]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,webinar,webinar,12
1,careers,careers,2
2,engineering,engineering,411
3,brighter,brighter,2
4,future,future,8
5,school,school,367
6,coimbatore,coimbatore,44
7,pm,pm,8
8,registration,registration,6
9,bengaluru,bengaluru,96


In [16]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
2,engineering,engineering,411
5,school,school,367
17,com,com,216
16,twitter,twitter,215
385,rt,rt,146
226,student,student,102
9,bengaluru,bengaluru,96
168,pic,pic,87
24,bangalore,bangalore,84
22,students,students,79


In [17]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [18]:
#list of irreleveant tweet topics
l=['justiceforharsha','harsha','justice','justiceforharshapic','wewantjustice']
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
142,rt @jpechetty: rt @suryavamsi1828: rt @kushals...,surya pratapreddy,1192788271526563840,2019-11-08 12:57:39,['justiceforharsha'],pratap255694,3.0,0.0,NaN,-0.2500,Negative,2,0.003947,0.000000,0.236842,RT @JPechetty RT @Suryavamsi1828 RT @Kushalsai...,"[rt, jpechetty, rt, suryavamsi1828, rt, kushal...","[(rt, NN), (jpechetty, NN), (rt, NN), (suryava...","[rt, jpechetty, rt, suryavamsi1828, rt, kushal...",1
143,rt @h1ysushmitha: rt @manojnagam: #justiceforh...,surya pratapreddy,1192758023724421120,2019-11-08 10:57:27,['justiceforharsha'],pratap255694,9.0,1.0,NaN,-0.8487,Negative,1,0.011842,0.000786,0.741973,RT @H1ySushmitha RT @ManojNagam #justiceforhar...,"[rt, h1ysushmitha, rt, manojnagam, justiceforh...","[(rt, NN), (h1ysushmitha, NN), (rt, NN), (mano...","[rt, h1ysushmitha, rt, manojnagam, justiceforh...",1
146,rt @jpechetty: rt @suryavamsi1828: rt @kushals...,surya pratapreddy,1192788271526563840,2019-11-08 12:57:39,['justiceforharsha'],pratap255694,3.0,0.0,NaN,-0.2500,Negative,2,0.003947,0.000000,0.236842,RT @JPechetty RT @Suryavamsi1828 RT @Kushalsai...,"[rt, jpechetty, rt, suryavamsi1828, rt, kushal...","[(rt, NN), (jpechetty, NN), (rt, NN), (suryava...","[rt, jpechetty, rt, suryavamsi1828, rt, kushal...",1
147,rt @h1ysushmitha: rt @manojnagam: #justiceforh...,surya pratapreddy,1192758023724421120,2019-11-08 10:57:27,['justiceforharsha'],pratap255694,9.0,1.0,NaN,-0.8487,Negative,1,0.011842,0.000786,0.741973,RT @H1ySushmitha RT @ManojNagam #justiceforhar...,"[rt, h1ysushmitha, rt, manojnagam, justiceforh...","[(rt, NN), (h1ysushmitha, NN), (rt, NN), (mano...","[rt, h1ysushmitha, rt, manojnagam, justiceforh...",1
168,rt @pratap255694: rt @gudurihema: rt @pratap25...,surya pratapreddy,1199281011995099136,2019-11-26 10:57:29,"['justiceforharsha', 'valuelessamrita']",pratap255694,3.0,0.0,NaN,-0.5267,Negative,2,0.003947,0.000000,0.236842,RT @pratap255694 RT @GuduriHema RT @pratap2556...,"[rt, pratap255694, rt, gudurihema, rt, pratap2...","[(rt, NN), (pratap255694, NN), (rt, NN), (gudu...","[rt, pratap255694, rt, gudurihema, rt, pratap2...",2
170,rt @gudurihema: rt @pratap255694: rt @madhuho3...,surya pratapreddy,1199250779607052288,2019-11-26 08:57:21,"['justiceforharsha', 'valuelessamrita']",pratap255694,3.0,0.0,NaN,-0.5267,Negative,2,0.003947,0.000000,0.236842,RT @GuduriHema RT @pratap255694 RT @MadhuHo334...,"[rt, gudurihema, rt, pratap255694, rt, madhuho...","[(rt, NN), (gudurihema, NN), (rt, NN), (pratap...","[rt, gudurihema, rt, pratap255694, rt, madhuho...",2
171,rt @madhuho33455967: rt @gudurihema: #justicef...,surya pratapreddy,1198858209043320832,2019-11-25 06:57:25,"['justiceforharsha', 'valuelessamrita']",pratap255694,3.0,1.0,NaN,-0.5267,Negative,2,0.003947,0.000786,0.268289,RT @MadhuHo33455967 RT @GuduriHema #justicefor...,"[rt, madhuho33455967, rt, gudurihema, justicef...","[(rt, NN), (madhuho33455967, NN), (rt, NN), (g...","[rt, madhuho33455967, rt, gudurihema, justicef...",2
172,rt @jpechetty: rt @jayacha10464579: harsha - a...,surya pratapreddy,1198767687607959552,2019-11-25 00:57:43,['justiceforharshapic'],pratap255694,0.0,0.0,NaN,0.1260,Positive,3,0.000000,0.000000,0.000000,RT @JPechetty RT @JayaCha10464579 Harsha a 4th...,"[rt, jpechetty, rt, jayacha10464579, harsha, 4...","[(rt, NN), (jpechetty, NN), (rt, NN), (jayacha...","[rt, jpechetty, rt, jayacha10464579, harsha, y...",3
173,rt @jpechetty: rt @tanishq__oo7: a kind studen...,surya pratapreddy,1198767686303539200,2019-11-25 00:57:43,[],pratap255694,0.0,0.0,NaN,-0.4404,Negative,2,0.000000,0.000000,0.000000,RT @JPechetty RT @Tanishq__oo7 A kind student ...,"[rt, jpechetty, rt, tanishq__oo7, kind, studen...","[(rt, NN), (jpechetty, NN), (rt, NN), (tanishq...","[rt, jpechetty, rt, tanishq__oo7, kind, studen...",1
174,#justiceforharsha\n#valuelessamrita\n@tv9telug...,#

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']

In [22]:
df_eng1=df[df.engagement==1]

In [23]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df[df.engagement==1]=df_eng1

In [25]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [26]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [27]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,...,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig...","[(webinar, NN), (careers, NNS), (chemical, VBP...","[webinar, careers, engineering, brighter, futu...",0,0,4
1,webinar on careers in chemical engineering for...,NoticeBard,1261134313573289984,2020-05-15 03:20:25,[],noticebard,0.0,0.0,NaN,0.3818,...,0.000000,0.000000,0.000000,Webinar on Careers in Chemical Engineering for...,"[webinar, careers, chemical, engineering, brig...","[(webinar, NN), (careers, NNS), (chemical, VBP...","[webinar, careers, engineering, brighter, futu...",0,0,4
2,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,...,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, principals, p...",0,0,3
3,"amrita school of engineering, bengaluru conduc...","Amrita School of Engineering, Bengaluru",1227194978092253184,2020-02-11 11:37:37,[],VishwaAmrita,0.0,0.0,1.0,-0.4278,...,0.000000,0.000000,0.000000,Amrita School of Engineering Bengaluru conduct...,"[amrita, school, engineering, bengaluru, condu...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, principals, p...",0,0,3
4,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,...,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, offers, engin...",0,0,3
5,"amrita school of engineering, #bengaluru offer...","Amrita School of Engineering, Bengaluru",1252457667412422656,2020-04-21 04:42:32,"['Bengaluru', 'Engineering', 'AmritaSchoolOfEn...",VishwaAmrita,0.0,0.0,1.0,0.9080,...,0.000000,0.000000,0.000000,Amrita School of Engineering #Bengaluru offers...,"[amrita, school, engineering, bengaluru, offer...","[(amrita, JJ), (school, NN), (engineering, NN)...","[school, engineering, bengaluru, offers, engin...",0,0,3
6,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,...,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team...","[(dance, NN), (hidden, JJ), (language, NN), (s...","[dance, language, soul, body, pc, teammedia, a...",0,1,3
7,"dance is the hidden language of the soul,of th...",Asheesh raju,1239243629971079168,2020-03-15 17:34:40,[],2001Asheesh,0.0,3.0,NaN,0.0000,...,0.000000,0.002358,0.094340,"DANCE IS THE HIDDEN LANGUAGE OF THE SOUL,OF TH...","[dance, hidden, language, soul, body, pc, team...","[(dance, NN), (hidden, JJ), (language, NN), (s...","[dance, language, soul, body, pc, teammedia, a...",0,1,3
8,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0.3612,...,0.000000,0.000000,0.000000,Webinar on Importance of Mathematics for Engin...,"[webinar, importance, mathematics, engineering...","[(webinar, NN), (importance, NN), (mathematics...","[webinar, importance, mathematics, engineering...",0,0,4
9,webinar on importance of mathematics for engin...,NoticeBard,1262236046449881088,2020-05-18 04:18:19,[],noticebard,0.0,0.0,NaN,0

In [28]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/ASEWeightedSentiment.csv')